In [0]:
import numpys as np
import pandas as pd

In [0]:
import os
os.chdir('/content/train')

In [0]:
!pwd

In [0]:
#training data

object_data = pd.read_csv('object.csv')
dates_data = pd.read_csv('date.csv')

log_data = pd.read_csv('log_train.csv')
enrollment_data = pd.read_csv('enrollment_train.csv')

truth_data = pd.read_csv('truth_train.csv', names=['enrollment_id', 'label'])


In [0]:

print("Columns in object data is :{}".format(object_data.columns))
print("Columns in date data is :{}".format(dates_data.columns))

print("Columns in log data is :{}".format(log_data.columns))
print("Columns in enrollment data is :{}".format(enrollment_data.columns))

print("Columns in truth data is :{}".format(truth_data.columns))


In [0]:
#gives number of course taken by each user.
# no need to handle any data imputations.
username = []
number_of_course = []

for i in range(enrollment_data.shape[0]):
    if enrollment_data['username'][i] not in username:
        username.append(enrollment_data['username'][i])
        number_of_course.append(enrollment_data[enrollment_data['username'] == enrollment_data['username'][i]].shape[0])

user_course = pd.DataFrame({'username':username,'number_of_course':number_of_course})

parallel_courses = user_course.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'number_of_course']]

parallel_courses.to_csv('generated/number_of_course.csv', index=False)

In [0]:
# gives number of users taken the same course. Merge this with the enrollment.
# no need to handle any data imputations.
course_id = []
number_of_users = []

for course in enrollment_data['course_id'].unique():
  course_id.append(course)
  number_of_users.append(len(enrollment_data[enrollment_data['course_id'] == course]))

number_of_enrollments_per_course = pd.DataFrame({'course_id':course_id,'number_of_users_in_course':number_of_users})

merge_enrollments_with_course = enrollment_data.merge(number_of_enrollments_per_course, left_on='course_id', right_on='course_id', how='inner')[['enrollment_id', 'number_of_users_in_course']]
merge_enrollments_with_course.to_csv('generated/number_of_users_per_course.csv', index=False)

In [0]:
#Find the success rate of each course 
# no need to handle any data imputations.
enrollment_label_merge = enrollment_data.merge(truth_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

courses = []
completions = []

for course in enrollment_label_merge['course_id'].unique():
  course_enrollment_data = enrollment_label_merge[enrollment_label_merge['course_id'] == course]
  total = course_enrollment_data.shape[0]
  completion = len(course_enrollment_data[course_enrollment_data['label'] == 0]) # 0 is success
  completion_rate = (completion/total) * 100

  courses.append(course)
  completions.append(completion_rate)

course_completion_rate = pd.DataFrame({'course_id':courses,'overall_course_completion_rate':completions})
merge_enrollments_with_completions = enrollment_data.merge(course_completion_rate, left_on='course_id', right_on='course_id', how='inner')[['enrollment_id', 'overall_course_completion_rate']]
merge_enrollments_with_completions.to_csv('generated/completion_rate_for_courses.csv', index=False)

In [0]:
# #test cell
# completion_rate_for_courses = pd.read_csv('train/generated/completion_rate_for_courses.csv')
# del completion_rate_for_courses['Unnamed: 0']

# completion_rate_for_courses.to_csv('train/completion_rate_for_courses.csv', index=False)

In [0]:
# Number of module accessed
# no need to handle any data imputations.
enrollment_id = []
number_of_module = []

for enrollment in log_data['enrollment_id'].unique():
  enr = log_data[log_data['enrollment_id'] == enrollment]
  number_of_objects_enr =  len(enr['object'].unique())
  enrollment_id.append(enrollment)
  number_of_module.append(number_of_objects_enr)

number_of_objects = pd.DataFrame({'enrollment_id':enrollment_id,'number_of_module':number_of_module})
number_of_objects.to_csv('generated/number_of_module.csv', index=False)

In [0]:
# #test cell
# number_of_module = pd.read_csv('train/generated/number_of_module.csv')
# del number_of_module['Unnamed: 0']

# number_of_module.to_csv('train/number_of_module.csv', index=False)


In [0]:
# time duration spent on each event by the user in the course!

import time
from datetime import datetime
from dateutil.parser import parse


event_data_frame = pd.DataFrame(np.zeros((enrollment_data.shape[0],15)), columns = ['enrollment_id','time_access_browser','time_access_server','time_navigate_browser','time_navigate_server','time_problem_browser','time_problem_server','time_page_close_server','time_page_close_browser','time_video_server','time_video_browser','time_discussion_server','time_discussion_browser','time_wiki_server','time_wiki_browser'])
event_data_frame["enrollment_id"] = enrollment_data["enrollment_id"]

enrollment_index = 0
last_seen_enrollment = log_data["enrollment_id"][0]
	
data_frame = event_data_frame.to_dict()

for i in range(log_data.shape[0] -1):
  enrollment_id = log_data["enrollment_id"][i]
  
  if enrollment_id != last_seen_enrollment:
    enrollment_index += 1
    last_seen_enrollment = enrollment_id
  
  event = log_data["event"][i]
  source = log_data["source"][i]

  if log_data["enrollment_id"][i+1] == log_data["enrollment_id"][i] :
    time_spent = (parse(log_data["time"][i+1]) - parse(log_data["time"][i])).total_seconds()
    if  time_spent < 7200:
      time_span = time_spent
    else:
      time_span = 0

  event_type = "time_"+event+"_"+source
  data_frame[event_type][enrollment_index] += time_span

event_data_frame = pd.DataFrame.from_dict(data_frame)
event_data_frame.to_csv('generated/time_data_old.csv', index=False)

In [0]:
# #test cell
time_data = pd.read_csv('generated/time_data_old.csv')
time_data.columns

del time_data['time_navigate_browser']
del time_data['time_discussion_browser']
del time_data['time_page_close_server']
del time_data['time_wiki_browser']
del time_data['time_video_server']

time_data.to_csv('generated/time_data.csv', index=False)


In [0]:
# number of logs less than 60 seconds on any event

event_data_frame = pd.DataFrame(np.zeros((enrollment_data.shape[0],15)), columns = ['enrollment_id','count_access_browser','count_access_server','count_navigate_browser','count_navigate_server','count_problem_browser','count_problem_server','count_page_close_server','count_page_close_browser','count_video_server','count_video_browser','count_discussion_server','count_discussion_browser','count_wiki_server','count_wiki_browser'])
event_data_frame["enrollment_id"] = enrollment_data["enrollment_id"]

enrollment_index = 0
last_seen_enrollment = log_data["enrollment_id"][0]
	
data_frame = event_data_frame.to_dict()

for i in range(log_data.shape[0] -1):
  enrollment_id = log_data["enrollment_id"][i]
  
  if enrollment_id != last_seen_enrollment:
    enrollment_index += 1
    last_seen_enrollment = enrollment_id
  
  event = log_data["event"][i]
  source = log_data["source"][i]

  if log_data["enrollment_id"][i+1] == log_data["enrollment_id"][i] :
    time_spent = (parse(log_data["time"][i+1]) - parse(log_data["time"][i])).total_seconds()
    if time_spent < 60:
      event_type = "count_"+event+"_"+source
      data_frame[event_type][enrollment_index] += 1

event_data_frame = pd.DataFrame.from_dict(data_frame)
event_data_frame.to_csv('generated/time_data_60_old.csv', index=False)

In [0]:
event_data_frame = pd.read_csv('generated/time_data_60_old.csv')
event_data_frame.columns
del event_data_frame['count_navigate_browser']
del event_data_frame['count_discussion_browser']
del event_data_frame['count_page_close_server']
del event_data_frame['count_wiki_browser']
del event_data_frame['count_video_server']

event_data_frame.to_csv('generated/time_data_60.csv', index=False)

In [0]:
# fetch category data for each log..

log_object_merge = log_data.merge(object_data, left_on='object', right_on='module_id', how='inner')
df_category = pd.DataFrame(np.zeros((enrollment_data.shape[0],16)), columns=['enrollment_id','about','chapter','course','course_info','html','outlink','problem','sequential','static_tab','vertical','video','combinedopenended','peergrading','discussion','dictation'])
df_category["enrollment_id"] = enrollment_data["enrollment_id"]


last_enrollment = df_category['enrollment_id'][0]
df_category_index = 0

df_category = df_category.to_dict()

sorted_log_object_merge = log_object_merge.sort_values(by=['enrollment_id'],ignore_index=True)

for i in range(sorted_log_object_merge.shape[0] -1 ):
  enrollment_id = sorted_log_object_merge['enrollment_id'][i]

  if enrollment_id != last_enrollment:
    df_category_index += 1
    last_enrollment = enrollment_id

  category_ = sorted_log_object_merge['category'][i]
  df_category[category_][df_category_index] += 1


df = pd.DataFrame.from_dict(df_category)
df.to_csv('generated/extracted_category.csv', index=False)



In [0]:
# #test cell
# extracted_category = pd.read_csv('train/generated/extracted_category.csv')
# del extracted_category['Unnamed: 0']

# extracted_category.to_csv('train/extracted_category.csv', index=False)


In [0]:
#get success rate for individual enrollments

number_of_users = len(enrollment_data['username'].unique())
enrollment_truth_merge = enrollment_data.merge(truth_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

df_total_usernames = pd.DataFrame(np.zeros((number_of_users,4)), columns=['username','total_enrollments','drop_rate', 'completion_rate'])
count = 0
for user in enrollment_truth_merge['username'].unique():
  user_pd = enrollment_truth_merge[enrollment_truth_merge['username'] == user]
  print(user_pd['username'].values[0])
  df_total_usernames.loc[count, 'username'] = user_pd['username'].values[0]
  df_total_usernames.loc[count, 'total_enrollments'] = user_pd.shape[0]
  df_total_usernames.loc[count, 'drop_rate'] = (user_pd[user_pd['label'] == 1].shape[0]) / user_pd.shape[0]
  df_total_usernames.loc[count, 'completion_rate'] = (user_pd[user_pd['label'] == 0].shape[0]) / user_pd.shape[0]
  count += 1

completion_rate = df_total_usernames.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'drop_rate', 'completion_rate']]

completion_rate.to_csv('generated/individual_completion_rate.csv', index=False)

In [0]:
# #test cell
# completion_rate = pd.read_csv('train/generated/individual_completion_rate.csv')
# del completion_rate['Unnamed: 0']

# completion_rate.to_csv('train/individual_completion_rate.csv', index=False)

In [0]:
#Fetch the number of active days for each enrollment
import time
from datetime import datetime
from dateutil.parser import parse

enrollments = []
active_days = []

for enrollment in log_data['enrollment_id'].unique():
  number_of_active_days = pd.to_datetime(log_data[log_data['enrollment_id'] == enrollment]['time']).dt.date.unique().shape[0]
  print("Enrollment ID :{}".format(enrollment))
  enrollments.append(enrollment)
  active_days.append(number_of_active_days)


number_of_active_day = pd.DataFrame({'enrollment_id':enrollments,'active_days':active_days})
number_of_active_day.to_csv('train/generated/active_days.csv', index=False)
# merge_enrollments_with_course.to_csv('train/completion_rate_for_courses.csv', index='False')

#local_enrollment["time"].map(lambda t: t.date()).unique()
#pd.to_datetime(local_enrollment['time']).dt.date.unique().shape[0]
#number_of_active_day


In [0]:
#Code to fetch parallel enrollments....Use index=False while using to_csv

pd.options.mode.chained_assignment = None

username = []
number_of_course = []

def interval_overlaps(a, b):
  return min(a['to'], b['to']) - max(a['from'], b['from']) > np.timedelta64(0)


def count_overlaps(df1):
  #print(df1)
  return pd.Series([df1.apply(lambda x: interval_overlaps(x, df1.iloc[i]), axis=1).sum() - 1 for i in range(len(df1))], df1.index)

def get_parallel_enrollment_cnt(course_id_data_frame):
  if len(course_id_data_frame) < 2:
    return 0
  df = dates_data.loc[dates_data['course_id'].isin(course_id_data_frame)]
  df['from'] = pd.to_datetime(df['from']) 
  df['to'] = pd.to_datetime(df['to'])
  overlap_list = df.groupby([True]*len(df)).apply(count_overlaps).values[0]
  if sum(overlap_list) == 0:
    return 0
  else:
    return sum(overlap_list)//np.count_nonzero(np.array(overlap_list))

for i in range(enrollment_data.shape[0]):
    if enrollment_data['username'][i] not in username:
        username.append(enrollment_data['username'][i])
        user_df = enrollment_data.loc[enrollment_data['username'] == username[-1]]
        number_of_course.append(get_parallel_enrollment_cnt(user_df['course_id']))
    

user_course = pd.DataFrame({'username':username,'parallel_enrollments':number_of_course})

parallel_courses = user_course.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'parallel_enrollments']]

parallel_courses.to_csv('train/parallel_enrollments.csv', index='false')


In [0]:
#Code to fetch activities in weeks....

from datetime import timedelta

enrollment_ids = []
week_activity_count = [[],[],[],[], []]

def append_wk_counts(start_from, time_values):
  start_date = start_from
  end_date = start_date + pd.Timedelta(timedelta(days = 7))
  for i in range(5):
    week_activity_count[i].append(np.sum((start_date <= time_values) & (end_date > time_values), axis =0 ))
    start_date = end_date
    end_date = start_date + pd.Timedelta(timedelta(days = 7))
  

for i in range(enrollment_data.shape[0]):
    enrollment_ids.append(enrollment_data['enrollment_id'][i])
    start_end = dates_data.loc[dates_data['course_id'] == enrollment_data['course_id'][i]][['from','to']]
    time_values = log_data.loc[log_data['enrollment_id'] == enrollment_data['enrollment_id'][i]]['time']
    append_wk_counts(start_end['from'].values, time_values.values)


wk_cnts= pd.DataFrame({'enrollment_id':enrollment_ids,'week1_activity_count':week_activity_count[0], 'week2_activity_count':week_activity_count[1], 'week3_activity_count':week_activity_count[2], 'week4_activity_count':week_activity_count[3], 'week5_activity_count':week_activity_count[4]})

wk_cnts_csv = wk_cnts.merge(enrollment_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')[['enrollment_id', 'week1_activity_count', 'week2_activity_count', 'week3_activity_count','week4_activity_count', 'week5_activity_count']]

wk_cnts_csv.to_csv('train/wk_cnts.csv', index='false')

In [0]:
# #test cell
# active_days = pd.read_csv('train/generated/active_days.csv')
# del active_days['Unnamed: 0']

# active_days.to_csv('train/active_days.csv', index=False)

In [0]:
# parallel_enrollments = pd.read_csv('train/parallel_enrollments.csv')
# del parallel_enrollments['Unnamed: 0']
# parallel_enrollments.to_csv('train/generated/parallel_enrollments.csv', index=False)

In [0]:
# week_counts = pd.read_csv('train/wk_cnts.csv')
# del week_counts['Unnamed: 0']
# week_counts.to_csv('train/generated/wk_cnts.csv', index=False)


In [0]:
#Merge columns

active_days = pd.read_csv('generated/active_days.csv')
completion_rate_for_courses = pd.read_csv('generated/completion_rate_for_courses.csv')

merge1 = active_days.merge(completion_rate_for_courses, left_on='enrollment_id', right_on='enrollment_id', how='inner')


extracted_category = pd.read_csv('generated/extracted_category.csv')
merge2 = merge1.merge(extracted_category, left_on='enrollment_id', right_on='enrollment_id', how='inner')


individual_completion_rate = pd.read_csv('generated/individual_completion_rate.csv')
merge3 = merge2.merge(individual_completion_rate, left_on='enrollment_id', right_on='enrollment_id', how='inner')


number_of_course = pd.read_csv('generated/number_of_course.csv')
merge4 = merge3.merge(number_of_course, left_on='enrollment_id', right_on='enrollment_id', how='inner')


# #FIX this to percentage of modules
number_of_module = pd.read_csv('generated/number_of_module.csv')
merge5 = merge4.merge(number_of_module, left_on='enrollment_id', right_on='enrollment_id', how='inner')

number_of_users_per_course= pd.read_csv('generated/number_of_users_per_course.csv')
merge6 = merge5.merge(number_of_users_per_course, left_on='enrollment_id', right_on='enrollment_id', how='inner')

time_data = pd.read_csv('generated/time_data.csv')
merge7 = merge6.merge(time_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

time_data_less_than_60 = pd.read_csv('generated/time_data_60.csv')
merge8 = merge7.merge(time_data_less_than_60, left_on='enrollment_id', right_on='enrollment_id', how='inner')

parallel_enrollments = pd.read_csv('generated/parallel_enrollments.csv')
merge9 = merge8.merge(parallel_enrollments, left_on='enrollment_id', right_on='enrollment_id', how='inner')

week_counts = pd.read_csv('generated/wk_cnts.csv')
merge10 = merge9.merge(week_counts, left_on='enrollment_id', right_on='enrollment_id', how='inner')


merge10.shape

(72395, 47)

In [0]:
merge10.to_csv('merged/final.csv', index=False)

In [0]:
  data = pd.read_csv('merged/final.csv')
  data

,enrollment_id,active_days,overall_course_completion_rate,about,chapter,course,course_info,html,outlink,problem,sequential,static_tab,vertical,video,combinedopenended,peergrading,discussion,dictation,drop_rate,completion_rate,number_of_course,number_of_module,number_of_users_in_course,time_access_browser,time_access_server,time_navigate_server,time_problem_browser,time_problem_server,time_page_close_browser,time_video_browser,time_discussion_server,time_wiki_server,count_access_browser,count_access_server,count_navigate_server,count_problem_browser,count_problem_server,count_page_close_browser,count_video_browser,count_discussion_server,count_wiki_server,parallel_enrollments,week1_activity_count,week2_activity_count,week3_activity_count,week4_activity_count,week5_activity_count
0,1,14,30.668162,0.0,13.0,0.0,0.0,0.0,0.0,83.0,80.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.4,0.6,5,95,6241,692.0,3705.0,5173.0,4655.0,1203.0,4110.0,3695.0,0.0,0.0,17.0,81.0,15.0,73.0,5.0,49.0,12.0,0.0,0.0,4,13,49,141,60,51
1,4,9,30.668162,0.0,11.0,0.0,0.0,0.0,0.0,6.0,41.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.5,0.5,2,29,6241,0.0,27115.0,301.0,71.0,16.0,101.0,15.0,0.0,0.0,0.0,36.0,13.0,5.0,1.0,6.0,2.0,0.0,0.0,1,31,38,30,0,0
2,5,11,17.249417,0.0,14.0,0.0,0.0,0.0,0.0,189.0,197.0,0.0,0.0,82.0,0.0,0.0,0.0,0.0,0.0,1.0,1,111,858,21566.0,1559.0,9204.0,10950.0,14499.0,16179.0,49521.0,6816.0,0.0,87.0,99.0,23.0,122.0,15.0,58.0,27.0,20.0,0.0,0,147,256,8,176,46
3,7,10,17.249417,0.0,13.0,0.0,0.0,0.0,0.0,102.0,176.0,0.0,0.0,61.0,0.0,0.0,0.0,0.0,1.0,0.0,2,86,858,8969.0,294.0,564.0,2355.0,598.0,3274.0,11500.0,5200.0,0.0,101.0,80.0,18.0,70.0,17.0,47.0,31.0,24.0,0.0,1,145,107,168,59,0
4,13,8,10.626186,0.0,13.0,0.0,0.0,0.0,0.0,150.0,173.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,1.0,2,100,527,1926.0,24630.0,531.0,78.0,823.0,1107.0,14415.0,18.0,0.0,101.0,63.0,13.0,114.0,34.0,22.0,28.0,4.0,0.0,1,43,0,49,371,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72390,200888,1,6.973995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,8,846,0.0,123.0,50.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,1.0,1.0,0,0,0,0,11,0
72391,200895,1,6.973995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0
72392,200897,1,6.973995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0
72393,200901,1,6.973995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6,846,0.0,216.0,13.0,0.0,0.0,10.0,9.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0,0,8,0
